In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

csv = pd.read_csv("../../../data/training_data/Martin_training.csv")
df = pd.DataFrame(csv)
df_time_offset = df["Timestamp"][0]
df["Timestamp"] = df["Timestamp"] - df_time_offset
df

,Timestamp,a_x,a_y,a_z,g_x,g_y,g_z,w_x,w_y,w_z,r_x,r_y,r_z,q_w,q_x,q_y,q_z
0,0.00,-0.303501,-0.752877,-0.633308,-71.657295,61.507023,51.717884,0.0,0.0,0.0,44.433311,-17.008606,170.083008,0.023430,-0.168715,-0.360720,-0.916988
1,0.01,-0.328947,-0.627887,-0.726913,-100.851082,-4.477046,40.885986,0.0,0.0,0.0,43.343395,-17.324430,170.359024,0.021782,-0.170147,-0.352022,-0.920139
2,0.02,-0.365706,-0.569115,-0.728412,-97.246567,-10.163084,35.955242,0.0,0.0,0.0,42.309643,-17.643412,170.563004,0.020651,-0.171876,-0.343646,-0.923006
3,0.03,-0.343222,-0.558287,-0.741697,-87.280083,-7.206237,28.493149,0.0,0.0,0.0,41.384247,-17.887228,170.735382,0.019880,-0.173148,-0.336165,-0.925536
4,0.04,-0.329618,-0.570622,-0.737472,-85.250969,-8.131433,23.945936,0.0,0.0,0.0,40.490299,-18.105560,170.869507,0.019472,-0.174355,-0.328893,-0.927928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,4.95,0.215725,-0.809689,-0.602411,28.053337,-0.656196,-29.528196,0.0,0.0,0.0,54.400913,9.542343,154.270065,0.234411,-0.029302,-0.460558,-0.855616
496,4.96,0.205720,-0.799891,-0.580055,31.868471,-6.377163,-25.625484,0.0,0.0,0.0,54.685543,9.714078,154.066574,0.236498,-0.029401,-0.462869,-0.853789
497,4.97,0.197790,-0.795772,-0.552438,31.457180,-7.189013,-21.422718,0.0,0.0,0.0,54.968708,9.847765,153.881714,0.238302,-0.029720,-0.465117,-0.852053
498,4.98,0.203719,-0.791840,-0.547220,27.802841,-5.424761,-17.323101,0.0,0.0,0.0,55.221657,9.958788,153.735931,0.239737,-0.029998,-0.467114,-0.850547


In [6]:
POSE_MAP= {
    "ingen_data": -1,
    "rett": 0,
    "framover": 1,
    "bakover": 2,
    "venstre": 3,
    "hoyre": 4,
    "overgang": 5
}

def get_timestamp_and_pose(filename):
    rows = []
    with open(filename, "r") as f:
        lines = f.readlines()
        offset = float(lines[1].strip().split(";")[1]) # This offset is the duration spent in the video before recording of data begun.
        for l in lines[1:]:
            sep_row = l.strip().split(";")
            finished_row = [round(float(x)-offset,2) for x in sep_row[1:3]]
            finished_row.append(POSE_MAP[sep_row[3].lower()])
            rows.append(finished_row)
    return rows

f_name = "../data/simon_data_2.txt"
stamped_poses = get_timestamp_and_pose(f_name)
print(stamped_poses)

df_stamped_poses = []
index = 0
for stamp in df["Timestamp"]:  
    pose_id = -1
    if stamp <= stamped_poses[-1][1]: 
        if stamp > stamped_poses[index][1]:
            index += 1
        pose_id = stamped_poses[index][2]
        # If timestamp is in new index, but haven't reached the starting time of this interval yet, set pose_id to default value 5.
        if stamp < stamped_poses[index][0]:
            pose_id = 5
    # If timestamp of the data point exceeds last timestamp recorded by the annotation, append default -1 value
    df_stamped_poses.append(pose_id)

#print(df_stamped_poses)

df["Pose"] = df_stamped_poses
#print(df[2400:2500][:])

#df['Pose'].hist(bins=4)
sns.lineplot(data=df, x="Timestamp", y='Pose')




NameError: name 'get_timestamp_and_pose' is not defined

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

X = df
y = df["Pose"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

pred = knn.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))


[[252   0   0   0   0   0]
 [  0 312   0   0   0   0]
 [  0   0 281   0   0   0]
 [  0   0   0 239   0   0]
 [  0   0   0   0 193   0]
 [  0   0   0   0   0 224]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       252
           1       1.00      1.00      1.00       312
           2       1.00      1.00      1.00       281
           3       1.00      1.00      1.00       239
           4       1.00      1.00      1.00       193
           5       1.00      1.00      1.00       224

    accuracy                           1.00      1501
   macro avg       1.00      1.00      1.00      1501
weighted avg       1.00      1.00      1.00      1501

